# K-Nearest Neighbor Classifer
k-nearest neighbors classifier can classify examples by assining them the class of the most similar labeled examples
simple but powerful
kNN is well suited for classification tasks where the relationship between the features are very complex and hard to understand
We have at training test set using the 

Lag # 1 | Lag # 2 | Lag # 3 | Direction

ingredients |  sweetness (feature) | crunchiness (feature) | type
    apple   |  10                  |      9                | fruit
    bacon   |  1                   |      4                | protein
    banana  |   2                  |      2                | fruit
    
Calculate the eculidian distance

dist(x,y) = sqrt((x1-y1)^2 + (x2-y2)^2 .... (xn-yn)^2)

k is a parameter when dealing with kNN: the algorithm uses the k closest training example in the feature space to make a decision

k =1 the smallest distance
k = 2 the 2 smallest distances
k =3 the 3 smallest differences

Choosing K values
- decieding how many neighbors to use for kNN-> determine how well the model will generalize and work on other datasets
- k is small -> noisy data or outliers have a huge impact on our classifer 
#### Called Overfitting
- k is large -> the classifier has the tendency to predict the majority class regardless of which neighbors are nearest 
#### Called Underfitting


#### kNN is a Lazy Learner, does not learn anything ahead of time only on fitting. There is no model builidng. 
Just store training data
this is a non parametric learning no parameters are to be learned about the data

In [13]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LogisticRegression
import pandas_datareader.data as web
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [14]:
stock_symbol="BA"
start_date="01/01/2017"
end_date="12/31/2019"
lags = 5

In [15]:
df = web.DataReader(stock_symbol, "yahoo", start_date,end_date)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-03,157.139999,155.210007,156.300003,156.970001,3426100.0,145.533676
2017-01-04,159.229996,157.809998,157.809998,158.619995,3189800.0,147.063477
2017-01-05,159.699997,157.399994,158.410004,158.710007,2796000.0,147.146927
2017-01-06,159.660004,157.889999,158.979996,159.100006,2426000.0,147.508514
2017-01-09,159.240005,158.169998,159.000000,158.320007,2839400.0,146.785309


In [16]:
#create new 
tslag = pd.DataFrame(index=df.index)
tslag["Today"]= df["Adj Close"]
tslag["Volume"] = df["Volume"]

#Create the shifted lag series of prior trading period close values
for i in range(0,lags):
    tslag["Lag%s" %str(i+1)] =  df["Adj Close"].shift(i+1)

# Create the returns data fram
dfret = pd.DataFrame(index=tslag.index)
dfret["Volume"] = tslag["Volume"]
dfret["Today"] = tslag["Today"].pct_change()*100.0
dfret.head()

,Volume,Today
Date,,
2017-01-03,3426100.0,NaN
2017-01-04,3189800.0,1.051166
2017-01-05,2796000.0,0.056744
2017-01-06,2426000.0,0.245732
2017-01-09,2839400.0,-0.490281


In [17]:
# Create the lagged percentage returns columns
for i in range(0,lags):
    dfret["Lag%s" % str(i+1)] = tslag["Lag%s" % str(i+1)].pct_change()*100.0

# Direction (+1 or -1 indicating an Up or Down down day)
dfret["Direction"] = np.sign(dfret["Today"])

#because of the shifts there are NaN avlues
dfret.drop(dfret.index[:5], inplace=True)
dfret.head()

C:\Users\Goliath\miniconda3\envs\quantitive_finance\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in sign
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Volume,Today,Lag1,Lag2,Lag3,Lag4,Lag5,Direction
Date,,,,,,,,
2017-01-10,2988300.0,0.473725,-0.490281,0.245732,0.056744,1.051166,NaN,1.0
2017-01-11,2416500.0,0.207443,0.473725,-0.490281,0.245732,0.056744,1.051166,1.0
2017-01-12,3020500.0,-0.696351,0.207443,0.473725,-0.490281,0.245732,0.056744,-1.0
2017-01-13,2435400.0,0.341145,-0.696351,0.207443,0.473725,-0.490281,0.245732,1.0
2017-01-17,2758800.0,-0.730330,0.341145,-0.696351,0.207443,0.473725,-0.490281,-1.0


In [29]:
X = dfret[["Lag1","Lag2","Lag3","Lag4"]]
y = dfret["Direction"]
start_test = datetime(2018,4,1)

X_train = X[X.index < start_test]
X_test = X[X.index >= start_test]
y_train = y[y.index < start_test]
y_test = y[y.index >= start_test]

#Instiantiate Model
k = 300
model = KNeighborsClassifier(k)

# Fit the model (not relevenat for kNN)
model.fit(X_train,y_train)


# Predict
prediction = model.predict(X_test)

# Score
model.score(X_test, y_test)

0.49095022624434387

In [30]:
#Confusion matrix
confusion_matrix(prediction, y_test)

array([[  0,   0,   0],
       [  0,   0,   0],
       [224,   1, 217]], dtype=int64)